# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("Weather_Output.csv")
cities.head()
cities=cities.dropna(axis=0, how='any')
cities


,Unnamed: 0,City,Lat,Lon,Cloud,Humidity,MaxTemp,Windspeed
0,0,aswan,24.0934,32.9070,0.0,10.0,100.36,7.78
1,1,airai,-8.9266,125.4092,59.0,90.0,62.13,1.86
2,2,cape town,-33.9258,18.4232,40.0,60.0,70.00,23.02
4,4,pinillos,8.9193,-74.4677,99.0,95.0,73.04,1.54
5,5,bluff,-46.6000,168.3333,47.0,90.0,55.00,5.01
...,...,...,...,...,...,...,...,...
74,74,provideniya,64.3833,-173.3000,100.0,86.0,-2.52,18.10
76,76,port alfred,-33.5906,26.8910,97.0,71.0,77.97,11.63
77,77,ushuaia,-54.8000,-68.3000,90.0,87.0,37.40,19.57
78,78,guerrero negro,27.9769,-114.0611,0.0,88.0,58.55,10.47


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
fig = gmaps.figure()
lat_lon = cities[["Lat","Lon"]]
pd.set_option('display.max_rows', cities.shape[0]+1)
heatmap = gmaps.heatmap_layer(lat_lon,weights=cities["Humidity"])
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
criteria_df = cities.loc[(cities["MaxTemp"]>70)&(cities["MaxTemp"]<80)&(cities["Cloud"]==0)]
criteria_df=criteria_df.dropna(axis=0, how='any')
criteria_df = criteria_df.drop(criteria_df.columns[[0]],axis=1)
criteria_df.head()


    Unnamed: 0       City      Lat       Lon  Cloud  Humidity  MaxTemp  \
39          39      adrar  20.5022  -10.0711    0.0      15.0    78.89   
53          53  esperance -33.8667  121.9000    0.0      57.0    75.20   

    Windspeed  
39      13.44  
53      12.66  


,City,Lat,Lon,Cloud,Humidity,MaxTemp,Windspeed
39,adrar,20.5022,-10.0711,0.0,15.0,78.89,13.44
53,esperance,-33.8667,121.9000,0.0,57.0,75.20,12.66


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
criteria_df['HotelNames'] = ""

for index,row in criteria_df.iterrows():
    lats = row['Lat']
    lons = row['Lon']
    
    params = {
        "location":f"{lats},{lons}",
        "radius":5000,
        "types":"hotel",
        "key":g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url,params=params).json()
    try:
        criteria_df.loc[index,"HotelNames"] = response['results'][0]['name']
    except:
        criteria_df.loc[index,"HotelNames"] = ""
    
criteria_df['HotelNames'].replace('',np.nan,inplace=True)
criteria_df.dropna(subset=['HotelNames'], inplace=True)
criteria_df

,City,Lat,Lon,Cloud,Humidity,MaxTemp,Windspeed,HotelNames
53,esperance,-33.8667,121.9,0.0,57.0,75.2,12.66,Esperance


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{HotelNames}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in criteria_df.iterrows()]
locations = criteria_df[["Lat", "Lon"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))